# **Limpieza de Datos**
El preprocesamiento de datos es una etapa fundamental en la construcción de cualquier modelo de predicción. En este notebook, aplicaremos técnicas para limpiar, transformar y preparar el dataset antes de su uso en el modelado.

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OrdinalEncoder
import matplotlib.pyplot as plt
import numpy as np

## **Carga del Dataset**
Cargamos el dataset desde un archivo CSV y realizamos una copia para evitar modificar los datos originales.

In [7]:
df = pd.read_csv('../data/Propensity.csv', index_col=0)
df.head()

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,...,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,COSTE_VENTA,km_anno,Mas_1_coche,Revisiones,Edad_Cliente,Tiempo
CODE,,,,,,,,,,,,,,,,,,,,,
CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,Asturias,...,Medio-Bajo,NO DATA,Averia muy grave,SI,2892,0,False,2,18,0
CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,Toledo,...,Medio-Bajo,SI,No,NO,1376,7187,False,2,53,0
CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,Lerida,...,Medio,NO DATA,No,NO,1376,0,True,4,21,3
CLI4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,Madrid,...,Medio,SI,Averia muy grave,SI,2015,7256,True,4,48,5
CLI5,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,Santa Cruz de Tenerife,...,Alto,NO DATA,No,NO,1818,0,True,3,21,3


## **1. Manejo de Valores Nulos**
Llenamos los valores faltantes en las variables numéricas con la mediana y en las categóricas con el modo más frecuente.

In [8]:
print("Valores nulos por columna antes de la imputación:")
print(df.isnull().sum())
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
df[num_cols] = df[num_cols].fillna(df[num_cols].median())
df.fillna(df.mode().iloc[0], inplace=True)

Valores nulos por columna antes de la imputación:
PRODUCTO               0
TIPO_CARROCERIA        0
COMBUSTIBLE            0
Potencia               0
TRANS                  0
FORMA_PAGO             0
ESTADO_CIVIL         890
GENERO               860
OcupaciOn              0
PROVINCIA              0
Campanna1              0
Campanna2              0
Campanna3              0
Zona_Renta         13178
REV_Garantia           0
Averia_grave           1
QUEJA_CAC              0
COSTE_VENTA            0
km_anno                0
Mas_1_coche            0
Revisiones             0
Edad_Cliente           0
Tiempo                 0
dtype: int64


## **2. Eliminación de Duplicados**
Eliminamos registros duplicados para evitar sesgos en el análisis.

In [9]:
df.duplicated().sum()

np.int64(151)

In [10]:
df.drop_duplicates(inplace = True)

In [11]:
df.shape[0]

57898

## **3. Conversión y Corrección de Tipos de Datos**


Eliminamos el único valor nulo de averia_grave

In [12]:
df = df[df['Averia_grave'].notna()]

Cambiamos por la moda en los valores necesarios

In [13]:
df[["ESTADO_CIVIL", "GENERO", "Zona_Renta"]] = df[["ESTADO_CIVIL", "GENERO", "Zona_Renta"]].apply(lambda col: col.fillna(col.mode()[0]))


## **4. Variables continuas y categóricas**
Aplicamos Label Encoding a columnas con muchas categorías y One-Hot Encoding a las de baja cardinalidad.

In [14]:
# Separar variables categóricas y continuas basado en el número de valores únicos y el tipo de dato
v_continuas = []
v_categoricas = []

for col in df.columns:
    if df[col].nunique() > 55 or df[col].dtype in ['float64', 'int64']:
        v_continuas.append(col)
    else:
        v_categoricas.append(col)

# Guardar los resultados en un diccionario
variables_separadas = {
    "Variables Continuas": v_continuas,
    "Variables Categóricas": v_categoricas
}

# Mostrar los resultados
print("Variables Continuas:", v_continuas)
print("Variables Categóricas:", v_categoricas)


Variables Continuas: ['COSTE_VENTA', 'km_anno', 'Revisiones', 'Edad_Cliente', 'Tiempo']
Variables Categóricas: ['PRODUCTO', 'TIPO_CARROCERIA', 'COMBUSTIBLE', 'Potencia', 'TRANS', 'FORMA_PAGO', 'ESTADO_CIVIL', 'GENERO', 'OcupaciOn', 'PROVINCIA', 'Campanna1', 'Campanna2', 'Campanna3', 'Zona_Renta', 'REV_Garantia', 'Averia_grave', 'QUEJA_CAC', 'Mas_1_coche']


### *Variables continuas*

In [15]:
df[v_continuas].head()

,COSTE_VENTA,km_anno,Revisiones,Edad_Cliente,Tiempo
CODE,,,,,
CLI1,2892,0,2,18,0
CLI2,1376,7187,2,53,0
CLI3,1376,0,4,21,3
CLI4,2015,7256,4,48,5
CLI5,1818,0,3,21,3


In [16]:
df[v_continuas].describe()

,COSTE_VENTA,km_anno,Revisiones,Edad_Cliente,Tiempo
count,57898.000000,57898.000000,57898.000000,57898.000000,57898.000000
mean,2540.763101,11833.851100,3.535873,47.361809,1.862810
std,1605.228908,10201.510238,2.527784,11.226005,3.093522
min,0.000000,0.000000,0.000000,18.000000,0.000000
25%,1595.000000,0.000000,2.000000,40.000000,0.000000
50%,2353.000000,11506.000000,3.000000,48.000000,0.000000
75%,3309.000000,17939.000000,5.000000,56.000000,4.000000
max,18455.000000,182331.000000,13.000000,71.000000,14.000000


El tratamiento de variables continuas es el conjunto de técnicas utilizadas para preparar y transformar variables numéricas que pueden tomar un rango infinito de valores dentro de un intervalo para su correcto uso en análisis de datos y modelos de Machine Learning.


In [17]:
def outliers_IQR(df, cols):
    outliers = {}
    for col in cols:
        Q1 = df[col].quantile(0.10)
        Q3 = df[col].quantile(0.90)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Contar valores atípicos
        outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col].count()
        # Eliminar outliers
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df, outliers

In [18]:
df, outliers_detectados = outliers_IQR(df, v_continuas)
print("Valores atípicos detectados:")
for col, count in outliers_detectados.items():
    print(f"- {col}: {count}")

Valores atípicos detectados:
- COSTE_VENTA: 37
- km_anno: 66
- Revisiones: 0
- Edad_Cliente: 0
- Tiempo: 0


In [19]:
df.shape[0] 

57795

### *Variables categóricas*

El tratamiento de variables categóricas es el proceso de transformar datos categóricos en un formato numérico adecuado para su uso en modelos de Machine Learning y análisis estadísticos. Esto se hace con diferentes estrategias según la naturaleza de cada variable.

In [20]:
df[v_categoricas].head()

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,Campanna1,Campanna2,Campanna3,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,Mas_1_coche
CODE,,,,,,,,,,,,,,,,,,
CLI1,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,M,Empresa,Asturias,SI,NO,NO,Medio-Bajo,NO DATA,Averia muy grave,SI,False
CLI2,A,TIPO1,FUEL 1,Baja,M,Contado,CASADO,F,Empresa,Toledo,NO,NO,NO,Medio-Bajo,SI,No,NO,False
CLI3,A,TIPO1,FUEL 1,Baja,M,Otros,CASADO,M,Empresa,Lerida,NO,NO,NO,Medio,NO DATA,No,NO,True
CLI4,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Empresa,Madrid,SI,NO,NO,Medio,SI,Averia muy grave,SI,True
CLI5,A,TIPO1,FUEL 1,Baja,M,Financiera Marca,CASADO,F,Funcionario,Santa Cruz de Tenerife,SI,NO,SI,Alto,NO DATA,No,NO,True


In [21]:
df[v_categoricas].nunique()

PRODUCTO           11
TIPO_CARROCERIA     8
COMBUSTIBLE         2
Potencia            3
TRANS               2
FORMA_PAGO          4
ESTADO_CIVIL        4
GENERO              2
OcupaciOn           3
PROVINCIA          53
Campanna1           2
Campanna2           2
Campanna3           2
Zona_Renta          4
REV_Garantia        2
Averia_grave        4
QUEJA_CAC           2
Mas_1_coche         2
dtype: int64

In [22]:
df[v_categoricas].describe()

,PRODUCTO,TIPO_CARROCERIA,COMBUSTIBLE,Potencia,TRANS,FORMA_PAGO,ESTADO_CIVIL,GENERO,OcupaciOn,PROVINCIA,Campanna1,Campanna2,Campanna3,Zona_Renta,REV_Garantia,Averia_grave,QUEJA_CAC,Mas_1_coche
count,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795,57795
unique,11,8,2,3,2,4,4,2,3,53,2,2,2,4,2,4,2,2
top,B,TIPO1,FUEL 2,Media,M,Contado,CASADO,M,Empresa,Madrid,NO,NO,NO,Alto,NO DATA,No,NO,False
freq,15968,23336,32922,39720,52734,30108,44190,40832,53020,10794,37292,50863,50935,30143,31842,29744,36065,40950


Antes de codificar las variables categóricas, es crucial revisar los valores únicos para detectar inconsistencias y errores. Esto permite corregir datos, agrupar categorías y definir la mejor estrategia de codificación (OrdinalEncoder o LabelEncoder) para garantizar un formato adecuado en el modelo predictivo.

### Codificación Ordinal (*OrdinalEncoder*)
Se usa cuando las categorías tienen un **orden lógico**, es decir, cuando existe una jerarquía entre los valores.  

 **Ejemplos en este caso:**
- **Potencia:** `"Baja" < "Media" < "Alta"`
- **Zona_Renta:** ` < "Otros"` "Medio-Bajo" < "Medio" < "Alto" 
- **Averia_grave:** `"No" < "Averia leve" < "Averia grave" < "Averia muy grave"`

 🔹 **Cómo funciona:**  
Cada categoría se convierte en un número según su nivel de importancia.  

 **Ejemplo:**  
```plaintext
["Baja", "Media", "Alta"] → [0, 1, 2]

In [23]:
# Definición del orden lógico de las variables ordinales.
ordinal_cols = {
    "Potencia": ["Baja", "Media", "Alta"],
    "Zona_Renta": ["Otros", "Medio-Bajo", "Medio", "Alto"],
    "Averia_grave": ["No", "Averia leve", "Averia grave", "Averia muy grave"]
}

In [24]:
ordinal_encoder = OrdinalEncoder(categories=[ordinal_cols[col] for col in ordinal_cols])
df[list(ordinal_cols.keys())] = ordinal_encoder.fit_transform(df[list(ordinal_cols.keys())])

### Codificación Nominal (*LabelEncoder*)

Se usa para variables **sin un orden jerárquico**, donde cada categoría es un grupo diferente, pero no tiene una relación de mayor o menor importancia.

 **Ejemplo:**
- **GENERO:** `"M"` y `"F"` se codifican como `[0, 1]`
- **PRODUCTO:** Cada tipo de producto recibe un número único.

 🔹 **Cómo funciona:**  
Cada categoría se convierte en un número único sin jerarquía.

**Ejemplo:**  
```plaintext
["Rojo", "Azul", "Verde"] → [0, 1, 2]


In [25]:
# Variables categóricas a codificar (solo restan las nominales)
label_cols = [col for col in v_categoricas if df[col].dtype != "float64"]

for i in label_cols: # Codificación de las variables categóricas nominales
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i])

## **6. Guardado del Dataset Preprocesado**
Guardamos el dataset limpio y transformado en un archivo CSV para su posterior uso en modelado.

In [26]:
df.to_csv("../data/Propensity_Processed.csv", index=False)